# Import Libraries

In [1]:
import pandas as pd # I/O files, data manipulation
import numpy as np

# Load Dataset

In [2]:
df = pd.read_csv('../data/all_phones.csv')

# Summary

In [3]:
num_samples = 3
summary_data = []

for column in df.columns:
    null_count = df[column].isnull().sum()
    unique_count = df[column].nunique()
    data_type = df[column].dtype
    min_value = df[column].min() if pd.api.types.is_numeric_dtype(data_type) else None
    max_value = df[column].max() if pd.api.types.is_numeric_dtype(data_type) else None
    mean = df[column].mean() if pd.api.types.is_numeric_dtype(data_type) else None
    std_dev = df[column].std() if pd.api.types.is_numeric_dtype(data_type) else None
    top_value = df[column].mode()[0] if pd.api.types.is_object_dtype(data_type) else None
    top_value_freq = df[column].value_counts().max() if pd.api.types.is_object_dtype(data_type) else None
    sample_values = df[column].sample(num_samples).tolist()
    
    summary_data.append([column, null_count, unique_count, data_type, min_value, max_value, mean, std_dev, top_value, top_value_freq, sample_values])

summary_df = pd.DataFrame(summary_data, columns=['Column', 'Null Count', 'Unique Count', 'Data Type', 'Min Value', 'Max Value', 'Mean', 'Std Dev', 'Top Value', 'Top Value Frequency', 'Sample Values'])
summary_df

,Column,Null Count,Unique Count,Data Type,Min Value,Max Value,Mean,Std Dev,Top Value,Top Value Frequency,Sample Values
0,phone_name,0,1512,object,NaN,NaN,NaN,NaN,Apple iPhone 11,1.0,"[Sony Xperia 10, Realme X2, Huawei Y7 Prime (2..."
1,brand,0,13,object,NaN,NaN,NaN,NaN,Xiaomi,264.0,"[Xiaomi, LG, Samsung]"
2,os,0,34,object,NaN,NaN,NaN,NaN,Android 11,322.0,"[Android 12, Android 10, Android 12]"
3,inches,0,99,object,NaN,NaN,NaN,NaN,"6.5""",171.0,"[6.58"", 5.84"", 6.72""]"
4,resolution,0,85,object,NaN,NaN,NaN,NaN,1080x2400 pixels,437.0,"[1080x2412 pixels, 1080x1920 pixels, 1080x2388..."
5,battery,0,136,int64,1821.0,7250.0,4389.798942,784.607022,None,NaN,"[2870, 2900, 4500]"
6,battery_type,1,2,object,NaN,NaN,NaN,NaN,Li-Po,1241.0,"[Li-Po, Li-Po, Li-Ion]"
7,ram,0,34,object,NaN,NaN,NaN,NaN,6/8GB RAM,212.0,"[1/2GB RAM, 4GB RAM, 2GB RAM]"
8,announcement_date,0,670,object,NaN,NaN,NaN,NaN,"2018, May",20.0,"[2020, April 09, 2023, February 27, 2021, June..."
9,weight,0,253,object,NaN,NaN,NaN,NaN,190 g (6.70 oz),52.0,"[191 g (6.74 oz), 169.5 g (6.00 oz), 199 g (7...."


#### As we can see, the dataset is in a very bad state. Incorrect data types should be fixed, feature engineering should be done, nan values should be dealt with etc.

# Data Cleaning and Preprocessing

- Fill nan values
- Feature extraction
- Change data types

#### - Fill nan values

We have only one missing value at 'battery_type' column.

In [4]:
df[df['battery_type'].isna()]

,phone_name,brand,os,inches,resolution,battery,battery_type,ram,announcement_date,weight,video,internal,price
243,Honor Magic5 Ultimate,Honor,Android 13,"6.81""",1312x2848 pixels,5450,NaN,16GB RAM,"2023, March 06",217 g (7.65 oz),"4K@30/60fps, 1080p@30/60fps, gyro-EIS, HDR10, ...",512GB 16GB RAM,About 900 EUR


Honor Magic 5 Ultimate has Li-Po battery type.

In [5]:
df['battery_type'].fillna('Li-Po', inplace=True)

#### - Feature Extraction

All values are in string format so we need to extract numeric values, format datetime, extract True False values etc.

In [6]:
#Phone names
df['phone_name'] = df['phone_name'].str.replace('Google ','')
df['phone_name'] = df['phone_name'].str.replace('Oppo ','')
df['phone_name'] = df['phone_name'].str.replace('Honor ','')
df['phone_name'] = df['phone_name'].str.replace('Lenovo ','')
df['phone_name'] = df['phone_name'].str.replace('Sony ','')
df['phone_name'] = df['phone_name'].str.replace('Apple ','')
df['phone_name'] = df['phone_name'].str.replace('Huawei ','')
df['phone_name'] = df['phone_name'].str.replace('OnePlus ','')
df['phone_name'] = df['phone_name'].str.replace('Xiaomi ','')
df['phone_name'] = df['phone_name'].str.replace('Samsung ','')
df['phone_name'] = df['phone_name'].str.replace('Vivo ','')
df['phone_name'] = df['phone_name'].str.replace('Realme ','')
df['phone_name'] = df['phone_name'].str.replace('LG ','')

In [7]:
#Inches
df['inches'].replace('["]','',regex=True, inplace=True)
df['inches'] = df['inches'].astype('float64')

In [8]:
#Resulotion
df['resolution'].replace(' pixels', '', regex=True, inplace=True)

In [9]:
#Ram
df['ram'] = df['ram'].str.findall(r'[0-9]+')
df['ram'] = df['ram'].apply(lambda x: max(x))
df.rename(columns={'ram':'ram(GB)'}, inplace=True)

In [10]:
#weight
df['weight'] = df['weight'].str.extract(r'([0-9]+)')
df['weight'] = df['weight'].astype('float64')
df.rename(columns={'weight':'weight(g)'}, inplace=True)
df['weight(g)'].fillna(df['weight(g)'].median(), inplace=True)

In [11]:
#video
df['video_720p'] = df['video'].str.contains('720p')
df['video_1080p'] = df['video'].str.contains('1080p')
df['video_4K'] = df['video'].str.contains('4K')
df['video_8K'] = df['video'].str.contains('8K')
df['video_30fps'] = df['video'].str.contains('30fps')
df['video_60fps'] = df['video'].str.contains('60fps')
df['video_120fps'] = df['video'].str.contains('120fps')
df['video_240fps'] = df['video'].str.contains('240fps')
df['video_480fps'] = df['video'].str.contains('480fps')
df['video_960fps'] = df['video'].str.contains('960fps')

df.loc[df['video_240fps'] == True, ['video_120fps','video_60fps','video_30fps']] = True
df.loc[df['video_4K'] == True, ['video_1080p','video_720p']] = True

df.drop(['video'], axis=1, inplace=True)

In [12]:
#internal (storage)
df['internal'] = df['internal'].str.findall(r'([0-9]+)')
df['internal'] = df['internal'].apply(lambda x: x[0])
df['internal'] = df['internal'].astype('int64')
df.rename(columns={'internal':'storage(GB)'}, inplace=True)

In [13]:
#price ($, £, €, ₹)
df['price'] = df['price'].str.replace('EUR','€')
df['price'] = df['price'].str.replace('INR','₹')
df['price'] = df['price'].apply(lambda x: x.split('/')[0])
df['price'] = df['price'].str.replace(u'\u2009','')
df['price'] = df['price'].str.replace(' ','')
df['price'] = df['price'].str.replace('About','')
df['price'] = df['price'].str.replace(',','')
df['price'].unique()

df.loc[df.price.str.contains('€'), 'currency'] = 'Euro'
df['price'] = df['price'].str.replace('€','')

df.loc[df.price.str.contains('$'), 'currency'] = 'Dolar'
df['price'] = df['price'].str.replace('$','')

df.loc[df.price.str.contains('₹'), 'currency'] = 'Rupi'
df['price'] = df['price'].str.replace('₹','')

df.loc[df.price.str.contains('£'), 'currency'] = 'Pound'
df['price'] = df['price'].str.replace('£','')

df['price'] = df['price'].astype('float64')

df.loc[df['currency'] == 'Euro', 'new_price'] = df['price'] * 1.07
df.loc[df['currency'] == 'Dolar', 'new_price'] = df['price'] * 1
df.loc[df['currency'] == 'Pound', 'new_price'] = df['price'] * 1.25
df.loc[df['currency'] == 'Rupi', 'new_price'] = df['price'] * 0.012

df.drop(['price','currency'], axis=1, inplace=True)
df.rename(columns={'new_price':'price(USD)'}, inplace=True)

#### - Change data types

In [14]:
#announcement_date
df['announcement_date'] = pd.to_datetime(df['announcement_date'], format='mixed', dayfirst=True)
df.sort_values('announcement_date', ascending=True, inplace=True)

# Write Cleaned Dataset to csv File

In [ ]:
df.to_csv('../data/cleaned_all_phones.csv', index=False)